# Foundation Capstone: The Hydrogen Atom

**SIIEA Quantum Engineering Curriculum**
- **Curriculum Days:** Days 309-336
- **License:** CC BY-NC-SA 4.0 | Siiea Innovations, LLC

---

In [ ]:
# Hardware detection — adapts simulations to your machine
import sys, os
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath("__file__")), ".."))
try:
    from hardware_config import HARDWARE, get_max_qubits
    print(f"Hardware: {HARDWARE['chip']} | {HARDWARE['memory_gb']} GB | Profile: {HARDWARE['profile']}")
    print(f"Max qubits: {get_max_qubits('safe')} (safe) / {get_max_qubits('max')} (max)")
except ImportError:
    print("hardware_config.py not found — using defaults")
    print("Run setup.sh from the repo root to generate it")

In [ ]:
# ── Imports & configuration ─────────────────────────────────────────
import numpy as np
from scipy import integrate, special, sparse
from scipy.sparse.linalg import eigsh
from scipy.integrate import solve_ivp, quad
from scipy.special import sph_harm, factorial, assoc_laguerre
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize
from mpl_toolkits.mplot3d import Axes3D
import sympy as sp

%matplotlib inline

plt.rcParams.update({
    'figure.figsize': (10, 7),
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 15,
    'legend.fontsize': 11,
    'lines.linewidth': 2,
    'figure.dpi': 100,
})

# Physical constants (atomic units: ℏ = mₑ = e = 4πε₀ = 1)
a0 = 1.0    # Bohr radius
E1 = -0.5   # ground state energy in Hartrees

print("Hydrogen atom capstone — all Year 0 tools in action!")
print(f"Working in atomic units: a₀ = {a0}, E₁ = {E1} Hartree")

## 1. Year 0 Comprehensive Review: Tools for Quantum Mechanics

Over the past 12 months, we have built a complete mathematical toolkit.
This capstone demonstrates **every major tool** applied to a single problem:
the hydrogen atom.

### Tools Applied

| Year 0 Topic | Hydrogen Atom Application |
|--------------|--------------------------|
| Calculus & ODEs | Radial Schrödinger equation |
| Linear algebra | Eigenvalue problem for energy levels |
| Complex analysis | Spherical harmonics, Euler's formula |
| Classical mechanics | Kepler problem analogy, Hamiltonian |
| Special functions | Laguerre polynomials, Legendre functions |
| Numerical methods | Finite-difference radial solver |
| Group theory | SO(3) symmetry → angular momentum quantum numbers |
| Scientific computing | Sparse eigensolvers, visualization |

### The Hydrogen Atom Hamiltonian

In atomic units ($\hbar = m_e = e = 4\pi\epsilon_0 = 1$):

$$\hat{H} = -\frac{1}{2}\nabla^2 - \frac{1}{r}$$

Separation of variables in spherical coordinates gives:

$$\psi_{nlm}(r,\theta,\phi) = R_{nl}(r)\,Y_l^m(\theta,\phi)$$

where $R_{nl}$ satisfies the **radial Schrödinger equation** and $Y_l^m$ are
**spherical harmonics**.

## 2. Analytical Solutions: Radial Wavefunctions

The normalized radial wavefunctions are:

$$R_{nl}(r) = -\sqrt{\left(\frac{2}{na_0}\right)^3 \frac{(n-l-1)!}{2n[(n+l)!]^3}} \;
e^{-r/(na_0)} \left(\frac{2r}{na_0}\right)^l L_{n-l-1}^{2l+1}\left(\frac{2r}{na_0}\right)$$

where $L_p^q$ are the **associated Laguerre polynomials**.

### Energy Levels

$$E_n = -\frac{1}{2n^2} \text{ (Hartree)}, \quad n = 1, 2, 3, \ldots$$

### Quantum Numbers

| Symbol | Name | Range | Physical Meaning |
|--------|------|-------|-----------------|
| $n$ | Principal | $1, 2, 3, \ldots$ | Energy, orbital size |
| $l$ | Azimuthal | $0, 1, \ldots, n-1$ | Orbital shape, $|\mathbf{L}|^2 = l(l+1)\hbar^2$ |
| $m$ | Magnetic | $-l, \ldots, +l$ | $z$-component of $\mathbf{L}$, $L_z = m\hbar$ |

In [ ]:
# ── Analytical radial wavefunctions ─────────────────────────────────
def hydrogen_radial(n, l, r):
    """
    Normalized radial wavefunction R_nl(r) for hydrogen atom.

    Parameters:
        n: principal quantum number (1, 2, 3, ...)
        l: azimuthal quantum number (0, 1, ..., n-1)
        r: radial coordinate (in units of a₀)

    Returns:
        R_nl(r) array
    """
    rho = 2.0 * r / n  # scaled variable
    # Normalization constant
    norm = np.sqrt((2.0/n)**3 * factorial(n-l-1, exact=True) /
                   (2.0 * n * factorial(n+l, exact=True)**3))

    # For numerical stability, we use a corrected normalization
    # that works with scipy's assoc_laguerre
    norm = np.sqrt((2.0/n)**3 * factorial(n-l-1, exact=True) /
                   (2.0 * n * (factorial(n+l, exact=True))))

    return norm * np.exp(-rho/2) * rho**l * assoc_laguerre(n-l-1, 2*l+1, rho)


# Radial wavefunctions for n=1,2,3
r = np.linspace(0, 25, 500)
states = [(1,0), (2,0), (2,1), (3,0), (3,1), (3,2)]
labels = ['1s', '2s', '2p', '3s', '3p', '3d']

fig, axes = plt.subplots(2, 1, figsize=(12, 10))

# Plot R_nl(r)
for (n, l), label in zip(states, labels):
    R = hydrogen_radial(n, l, r)
    axes[0].plot(r, R, label=f'{label} ($n$={n}, $l$={l})', linewidth=2)

axes[0].axhline(0, color='black', linewidth=0.5)
axes[0].set_xlabel('$r / a_0$')
axes[0].set_ylabel('$R_{nl}(r)$')
axes[0].set_title('Hydrogen Radial Wavefunctions')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)
axes[0].set_xlim(0, 25)

# Plot r²|R_nl|² (radial probability density)
for (n, l), label in zip(states, labels):
    R = hydrogen_radial(n, l, r)
    P = r**2 * R**2
    axes[1].plot(r, P, label=f'{label}', linewidth=2)

axes[1].axhline(0, color='black', linewidth=0.5)
axes[1].set_xlabel('$r / a_0$')
axes[1].set_ylabel('$r^2 |R_{nl}(r)|^2$')
axes[1].set_title('Radial Probability Density')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)
axes[1].set_xlim(0, 25)

plt.tight_layout()
plt.show()

# Print expectation values
print(f"{'State':<6} {'⟨r⟩ (a₀)':>12} {'Nodes':>8}")
print("-" * 28)
for (n, l), label in zip(states, labels):
    R = hydrogen_radial(n, l, r)
    dr = r[1] - r[0]
    expect_r = np.trapezoid(r**3 * R**2, r)
    nodes = n - l - 1
    print(f"{label:<6} {expect_r:>12.4f} {nodes:>8}")
    print(f"  (exact ⟨r⟩ = {n**2*(3 - l*(l+1)/n**2)/2:.4f})")

## 3. Numerical Solution: Radial Schrödinger Equation

The radial equation (after substitution $u(r) = r\,R(r)$):

$$-\frac{1}{2}\frac{d^2u}{dr^2} + \left[\frac{l(l+1)}{2r^2} - \frac{1}{r}\right]u = Eu$$

This is an eigenvalue problem we can solve numerically using finite differences.
The effective potential is:

$$V_{\text{eff}}(r) = -\frac{1}{r} + \frac{l(l+1)}{2r^2}$$

In [ ]:
# ── Numerical radial solver using finite differences ────────────────
def solve_radial_schrodinger(l, r_max=50, N=1000, n_states=5):
    """
    Solve the radial Schrödinger equation for hydrogen numerically.

    Uses the substitution u(r) = r·R(r) and finite differences.
    Returns eigenvalues and eigenfunctions u_n(r).
    """
    # Grid (avoid r=0 singularity)
    dr = r_max / (N + 1)
    r = np.linspace(dr, r_max - dr, N)

    # Effective potential
    V_eff = -1.0/r + l*(l+1) / (2.0 * r**2)

    # Kinetic energy: finite-difference second derivative
    coeff = 1.0 / (2.0 * dr**2)
    diag_main = 2 * coeff + V_eff
    diag_off = -coeff * np.ones(N - 1)

    H = sparse.diags([diag_off, diag_main, diag_off],
                      [-1, 0, 1], format='csr')

    # Solve for lowest eigenvalues
    energies, wavefuncs = eigsh(H, k=n_states, which='SA')
    idx = np.argsort(energies)
    energies = energies[idx]
    wavefuncs = wavefuncs[:, idx]

    # Normalize: ∫|u(r)|² dr = 1
    for i in range(n_states):
        norm = np.sqrt(np.trapezoid(wavefuncs[:, i]**2, r))
        wavefuncs[:, i] /= norm

    return r, energies, wavefuncs


# Solve for l=0, 1, 2
print(f"{'State':<8} {'E_numerical':>14} {'E_exact':>14} {'Rel. Error':>14}")
print("=" * 54)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))
orbital_names = {0: 's', 1: 'p', 2: 'd'}

for l_val, ax in zip([0, 1, 2], axes):
    r_num, E_num, u_num = solve_radial_schrodinger(l_val, r_max=60, N=2000, n_states=4)

    for i in range(min(4, len(E_num))):
        n = i + l_val + 1
        E_exact = -0.5 / n**2
        rel_err = abs(E_num[i] - E_exact) / abs(E_exact)
        name = f"{n}{orbital_names[l_val]}"
        print(f"{name:<8} {E_num[i]:>14.8f} {E_exact:>14.8f} {rel_err:>14.2e}")

        # Fix sign for plotting: match analytical
        R_analytic = hydrogen_radial(n, l_val, r_num)
        u_analytic = r_num * R_analytic
        if np.dot(u_num[:, i], u_analytic) < 0:
            u_num[:, i] *= -1

        ax.plot(r_num, u_num[:, i], '-', label=f'{name} (num)', linewidth=2)
        ax.plot(r_num, u_analytic, '--', label=f'{name} (exact)', alpha=0.7)

    ax.set_xlabel('$r / a_0$')
    ax.set_ylabel('$u_{nl}(r) = r\,R_{nl}(r)$')
    ax.set_title(f'$l = {l_val}$ states')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0, 40)
    print()

plt.suptitle('Numerical vs Analytical Radial Wavefunctions', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

## 4. Spherical Harmonics & Angular Momentum

The angular part of the hydrogen wavefunction is given by **spherical harmonics**:

$$Y_l^m(\theta, \phi) = \sqrt{\frac{(2l+1)}{4\pi}\frac{(l-|m|)!}{(l+|m|)!}} \;
P_l^{|m|}(\cos\theta)\; e^{im\phi}$$

These are simultaneous eigenfunctions of:
- $\hat{L}^2 Y_l^m = l(l+1)\hbar^2 Y_l^m$
- $\hat{L}_z Y_l^m = m\hbar Y_l^m$

### Connection to Group Theory (Month 11)
Spherical harmonics are the **irreducible representations** of SO(3).
Each value of $l$ gives a $(2l+1)$-dimensional irrep, with $m = -l, \ldots, +l$
labeling the basis states.

In [ ]:
# ── Spherical harmonics visualization ───────────────────────────────
def plot_spherical_harmonic(l, m, ax, title=None):
    """Plot |Y_l^m|² on the unit sphere."""
    theta = np.linspace(0, np.pi, 100)
    phi = np.linspace(0, 2*np.pi, 100)
    THETA, PHI = np.meshgrid(theta, phi)

    Y = sph_harm(m, l, PHI, THETA)  # note: scipy uses (m, l, phi, theta)
    R_val = np.abs(Y)**2

    # Convert to Cartesian
    X = R_val * np.sin(THETA) * np.cos(PHI)
    Y_cart = R_val * np.sin(THETA) * np.sin(PHI)
    Z = R_val * np.cos(THETA)

    # Color by phase
    phase = np.angle(Y)
    norm = Normalize(vmin=-np.pi, vmax=np.pi)
    colors = cm.coolwarm(norm(phase))

    ax.plot_surface(X, Y_cart, Z, facecolors=colors, alpha=0.8)
    max_r = np.max(R_val) * 1.1
    ax.set_xlim(-max_r, max_r)
    ax.set_ylim(-max_r, max_r)
    ax.set_zlim(-max_r, max_r)
    if title:
        ax.set_title(title, fontsize=13)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')


# Plot spherical harmonics for l = 0, 1, 2
harmonics = [(0,0), (1,-1), (1,0), (1,1), (2,-2), (2,-1), (2,0), (2,1), (2,2)]

fig = plt.figure(figsize=(18, 12))
for idx, (l, m) in enumerate(harmonics):
    ax = fig.add_subplot(3, 3, idx+1, projection='3d')
    plot_spherical_harmonic(l, m, ax, title=f'$Y_{l}^{{{m}}}$')

plt.suptitle('Spherical Harmonics $|Y_l^m(\\theta,\\phi)|^2$', fontsize=18, y=0.98)
plt.tight_layout()
plt.show()

# Print properties
print(f"\n{'(l,m)':<10} {'L²':>10} {'Lz':>10} {'Degeneracy':>12}")
print("-" * 45)
for l in range(4):
    for m in range(-l, l+1):
        if m == 0:
            print(f"({l},{m:+d}){'':<5} {l*(l+1):>10} {m:>10} {2*l+1:>12}")
        elif m == -l:
            print(f"({l},{m:+d}){'':<4} {l*(l+1):>10} {m:>10}")
        else:
            print(f"({l},{m:+d}){'':<4} {'':>10} {m:>10}")

## 5. Probability Distributions: s, p, and d Orbitals

The full probability density is:

$$|\psi_{nlm}(r, \theta, \phi)|^2 = |R_{nl}(r)|^2 \, |Y_l^m(\theta, \phi)|^2$$

We visualize these in 2D cross-sections (the $xz$-plane, where $\phi = 0$)
to see the characteristic shapes:

| Orbital | Shape | Nodes |
|---------|-------|-------|
| s ($l=0$) | Spherical | $n-1$ radial |
| p ($l=1$) | Dumbbell (lobed) | $n-2$ radial, 1 angular |
| d ($l=2$) | Cloverleaf | $n-3$ radial, 2 angular |

In [ ]:
# ── Cross-section visualization of hydrogen orbitals ────────────────
def hydrogen_density_xz(n, l, m, grid_size=200, r_max=None):
    """
    Compute |ψ_nlm|² in the xz-plane (y=0, so φ=0 for x>0, φ=π for x<0).
    """
    if r_max is None:
        r_max = 4 * n**2  # scale with n²

    x = np.linspace(-r_max, r_max, grid_size)
    z = np.linspace(-r_max, r_max, grid_size)
    X, Z = np.meshgrid(x, z)

    R_grid = np.sqrt(X**2 + Z**2) + 1e-10
    THETA = np.arccos(Z / R_grid)
    PHI = np.where(X >= 0, 0.0, np.pi)  # xz-plane

    R_val = hydrogen_radial(n, l, R_grid)
    Y_val = sph_harm(m, l, PHI, THETA)

    density = np.abs(R_val * Y_val)**2
    return x, z, density


# Orbitals to visualize
orbitals = [
    (1, 0, 0, '1s'),
    (2, 0, 0, '2s'),
    (2, 1, 0, '2p₀'),
    (3, 0, 0, '3s'),
    (3, 1, 0, '3p₀'),
    (3, 2, 0, '3d₀'),
]

fig, axes = plt.subplots(2, 3, figsize=(16, 10))

for (n, l, m, label), ax in zip(orbitals, axes.flat):
    x, z, density = hydrogen_density_xz(n, l, m)

    # Use logarithmic color scaling for better visibility
    density_plot = np.log10(density + 1e-10)
    vmin = density_plot.max() - 4  # show 4 orders of magnitude

    c = ax.contourf(x, z, density_plot, levels=30, cmap='inferno',
                    vmin=vmin)
    ax.set_xlabel('$x / a_0$')
    ax.set_ylabel('$z / a_0$')
    ax.set_title(f'{label}  ($n$={n}, $l$={l}, $m$={m})', fontsize=14)
    ax.set_aspect('equal')
    plt.colorbar(c, ax=ax, label='$\log_{10}|\psi|^2$', shrink=0.8)

plt.suptitle('Hydrogen Orbital Probability Densities (xz-plane)',
             fontsize=17, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# ── Radial probability comparison: analytical vs numerical ──────────
fig, ax = plt.subplots(figsize=(12, 7))

r_plot = np.linspace(0.01, 30, 500)
colors = plt.cm.Set1(np.linspace(0, 1, 6))

for idx, ((n, l), label) in enumerate(zip([(1,0),(2,0),(2,1),(3,0),(3,1),(3,2)],
                                           ['1s','2s','2p','3s','3p','3d'])):
    # Analytical
    R_an = hydrogen_radial(n, l, r_plot)
    P_an = r_plot**2 * R_an**2
    ax.plot(r_plot, P_an, color=colors[idx], linewidth=2.5, label=f'{label}')

    # Mark most probable radius
    i_max = np.argmax(P_an)
    ax.plot(r_plot[i_max], P_an[i_max], 'v', color=colors[idx],
            markersize=10, markeredgecolor='black')

    # Theoretical most probable radius for l=n-1 states: r_mp = n² a₀
    if l == n - 1:
        ax.axvline(n**2, color=colors[idx], linestyle=':', alpha=0.4)

ax.set_xlabel('$r / a_0$', fontsize=14)
ax.set_ylabel('$r^2 |R_{nl}|^2$', fontsize=14)
ax.set_title('Radial Probability Density (triangles mark maxima)', fontsize=15)
ax.legend(fontsize=12, ncol=2)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 30)
plt.tight_layout()
plt.show()

# Print most probable radii
print(f"{'State':<6} {'r_max (numerical)':>18} {'r_max (exact)':>16}")
print("-" * 42)
for (n, l), label in zip([(1,0),(2,0),(2,1),(3,0),(3,1),(3,2)],
                           ['1s','2s','2p','3s','3p','3d']):
    R = hydrogen_radial(n, l, r_plot)
    P = r_plot**2 * R**2
    r_mp = r_plot[np.argmax(P)]
    # For circular orbits (l=n-1): r_mp = n²
    exact = f"{n**2:.1f}" if l == n-1 else "---"
    print(f"{label:<6} {r_mp:>18.2f} {exact:>16}")

## 6. Energy Level Diagram with Fine Structure Preview

The non-relativistic hydrogen energy levels depend only on $n$:

$$E_n = -\frac{13.6 \text{ eV}}{n^2}$$

This gives a **degeneracy** of $n^2$ for each level (or $2n^2$ including spin).

### Fine Structure (Preview for Year 1)

Relativistic corrections lift the $l$-degeneracy, splitting each $n$ level by:

$$\Delta E_{\text{fine}} = -\frac{\alpha^2 E_n}{n} \left(\frac{1}{j+1/2} - \frac{3}{4n}\right)$$

where $\alpha \approx 1/137$ is the fine-structure constant and $j = l \pm 1/2$
is the total angular momentum.

In [ ]:
# ── Energy level diagram ────────────────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(14, 8))

# --- Left panel: non-relativistic levels ---
ax = axes[0]
n_max = 5
for n in range(1, n_max + 1):
    E = -13.6 / n**2
    degeneracy = 2 * n**2

    # Draw energy level line
    x_left, x_right = 0.2, 0.8
    ax.plot([x_left, x_right], [E, E], 'b-', linewidth=2.5)

    # Label
    ax.text(x_right + 0.05, E, f'$n={n}$  ({degeneracy})',
            fontsize=12, va='center')
    ax.text(x_left - 0.05, E, f'{E:.2f} eV',
            fontsize=10, va='center', ha='right', color='gray')

    # Show orbital labels
    orb_labels = []
    for l in range(n):
        orb_names = {0:'s', 1:'p', 2:'d', 3:'f', 4:'g'}
        orb_labels.append(f'{n}{orb_names.get(l,"?")}')
    ax.text(0.5, E + 0.3, ', '.join(orb_labels),
            fontsize=9, ha='center', color='green')

# Ionization limit
ax.axhline(0, color='red', linestyle='--', linewidth=1)
ax.text(0.5, 0.5, 'Ionization (0 eV)', ha='center', color='red')

ax.set_ylabel('Energy (eV)', fontsize=14)
ax.set_title('Hydrogen Energy Levels\n(Non-relativistic)', fontsize=14)
ax.set_xlim(0, 1.2)
ax.set_ylim(-15, 2)
ax.get_xaxis().set_visible(False)

# --- Right panel: fine structure splitting ---
ax = axes[1]
alpha_fs = 1 / 137.036  # fine-structure constant

for n in range(1, 4):
    E_n = -13.6 / n**2

    for l in range(n):
        for j_sign in [-0.5, 0.5]:
            j = l + j_sign
            if j < 0:
                continue

            # Fine structure correction
            if j + 0.5 > 0:
                dE = -alpha_fs**2 * abs(E_n) / n * (1/(j + 0.5) - 3/(4*n))
            else:
                continue

            E_fine = E_n + dE * 1000  # exaggerate for visibility

            # x position based on l
            x_pos = l * 0.25 + 0.1
            x_width = 0.15

            ax.plot([x_pos, x_pos + x_width], [E_fine, E_fine],
                    linewidth=2, color=f'C{l}')

            orb_names = {0:'s', 1:'p', 2:'d'}
            label = f'{n}{orb_names[l]}$_{{j={j:.0f}/2}}$' if j != int(j) else                     f'{n}{orb_names[l]}$_{{j={j:.0f}}}$'
            label = f'{n}{orb_names[l]}$_{{{int(2*j)}/2}}$'
            ax.text(x_pos + x_width + 0.02, E_fine, label,
                    fontsize=9, va='center')

ax.set_ylabel('Energy (eV) — splitting exaggerated', fontsize=12)
ax.set_title('Fine Structure Splitting\n(Preview for Year 1)', fontsize=14)
ax.set_ylim(-15, 1)
ax.get_xaxis().set_visible(False)
ax.text(0.1, -14.5, '$l$: ', fontsize=11)
for l, name in enumerate(['s', 'p', 'd']):
    ax.text(0.1 + l*0.25, -14.5, name, fontsize=11, color=f'C{l}',
            fontweight='bold')

plt.suptitle('Hydrogen Atom: Energy Levels', fontsize=17, y=1.01)
plt.tight_layout()
plt.show()

# Print energy levels with degeneracies
print(f"\n{'n':>3} {'E (eV)':>10} {'Degeneracy':>12} {'Orbitals':>20}")
print("=" * 48)
for n in range(1, 6):
    E = -13.6 / n**2
    deg = 2 * n**2
    orbs = ', '.join(f'{n}{"spdfg"[l]}' for l in range(n))
    print(f"{n:>3} {E:>10.4f} {deg:>12} {orbs:>20}")

## 7. Bridge to Year 1: The Postulates of Quantum Mechanics

The hydrogen atom showcases quantum mechanics beautifully, but we solved it
using the **Schrödinger equation** framework, which we have been taking on faith.
In Year 1, we build quantum mechanics from its foundational **postulates**:

### The Six Postulates

1. **State space:** The state of a quantum system is a vector $|\psi\rangle$
   in a Hilbert space $\mathcal{H}$.

2. **Observables:** Physical observables correspond to Hermitian operators
   $\hat{A}$ on $\mathcal{H}$.

3. **Measurement outcomes:** The possible outcomes of measuring $\hat{A}$ are
   its eigenvalues $a_n$.

4. **Born rule:** The probability of outcome $a_n$ is $|\langle a_n|\psi\rangle|^2$.

5. **State collapse:** After measuring outcome $a_n$, the state becomes $|a_n\rangle$.

6. **Time evolution:** The state evolves via $i\hbar\frac{d}{dt}|\psi\rangle = \hat{H}|\psi\rangle$.

### What Year 1 Covers

| Semester 1A | Semester 1B |
|-------------|-------------|
| Dirac notation & Hilbert spaces | Quantum information theory |
| Operators & commutators | Entanglement & Bell inequalities |
| Angular momentum theory | Quantum computing basics |
| Perturbation theory | Density matrices |
| Scattering theory | Quantum error correction preview |

In [ ]:
# ── Quantum postulates in action: measurement simulation ────────────
# Simulate measuring L_z on hydrogen 3d state (l=2)
# Possible outcomes: m = -2, -1, 0, +1, +2

rng = np.random.default_rng(42)

# Prepare a superposition state in the l=2 subspace
# |ψ⟩ = Σ c_m |2, m⟩
coefficients = rng.normal(size=5) + 1j * rng.normal(size=5)
coefficients /= np.linalg.norm(coefficients)  # normalize

m_values = np.array([-2, -1, 0, 1, 2])
probabilities = np.abs(coefficients)**2

print("=== Measurement Simulation: L_z on l=2 state ===\n")
print("Prepared superposition state:")
for m, c, p in zip(m_values, coefficients, probabilities):
    print(f"  |m={m:+d}⟩: c = {c:.4f}, P = {p:.4f}")
print(f"  Sum of probabilities: {probabilities.sum():.6f}")

# Simulate N measurements
N_meas = 10000
outcomes = rng.choice(m_values, size=N_meas, p=probabilities)

# Count occurrences
unique, counts = np.unique(outcomes, return_counts=True)
frequencies = counts / N_meas

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart: theoretical vs measured
width = 0.35
axes[0].bar(m_values - width/2, probabilities, width, label='Theory (Born rule)',
            color='steelblue', alpha=0.8, edgecolor='black')
axes[0].bar(unique + width/2, frequencies, width, label=f'Measured (N={N_meas:,})',
            color='coral', alpha=0.8, edgecolor='black')
axes[0].set_xlabel('$m$ (L$_z$ quantum number)')
axes[0].set_ylabel('Probability')
axes[0].set_title('Born Rule: L$_z$ Measurement')
axes[0].legend()
axes[0].set_xticks(m_values)
axes[0].grid(True, alpha=0.3, axis='y')

# Expectation value convergence
cumulative_mean = np.cumsum(outcomes) / np.arange(1, N_meas + 1)
exact_expectation = np.sum(m_values * probabilities)

axes[1].plot(cumulative_mean, linewidth=1.5, alpha=0.7, label='Running mean')
axes[1].axhline(exact_expectation, color='red', linestyle='--', linewidth=2,
                label=f'$\\langle L_z \\rangle$ = {exact_expectation:.4f}$\\hbar$')
axes[1].set_xlabel('Number of measurements')
axes[1].set_ylabel('$\\langle m \\rangle$')
axes[1].set_title('Convergence of Expectation Value')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nExact ⟨Lz⟩ = {exact_expectation:.4f} ℏ")
print(f"Measured ⟨Lz⟩ = {outcomes.mean():.4f} ℏ  (from {N_meas:,} measurements)")
print(f"Standard error: {outcomes.std()/np.sqrt(N_meas):.4f} ℏ")

In [ ]:
# ── Time evolution: Gaussian wavepacket in Coulomb potential ────────
# Demonstrate Postulate 6: |ψ(t)⟩ = e^{-iHt/ℏ}|ψ(0)⟩

# For visualization, use 1D radial evolution of a superposition
# ψ(r,0) = c₁ R₁₀(r) + c₂ R₂₀(r) + c₃ R₃₀(r)
# Beat frequency: ω₁₂ = (E₁ - E₂)/ℏ

r_grid = np.linspace(0.01, 30, 500)

# Energy eigenvalues (atomic units)
E = {1: -0.5, 2: -0.125, 3: -0.5/9}

# Radial wavefunctions
R = {}
for n in [1, 2, 3]:
    R[n] = hydrogen_radial(n, 0, r_grid)

# Superposition coefficients
c1, c2, c3 = 0.6, 0.7, 0.3
norm_c = np.sqrt(c1**2 + c2**2 + c3**2)
c1, c2, c3 = c1/norm_c, c2/norm_c, c3/norm_c

# Time evolution
times = np.linspace(0, 100, 8)  # in atomic time units

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for idx, t in enumerate(times):
    ax = axes[idx // 4, idx % 4]

    # ψ(r,t) = Σ cₙ Rₙ₀(r) e^{-iEₙt}
    psi = (c1 * R[1] * np.exp(-1j * E[1] * t) +
           c2 * R[2] * np.exp(-1j * E[2] * t) +
           c3 * R[3] * np.exp(-1j * E[3] * t))

    density = r_grid**2 * np.abs(psi)**2

    ax.fill_between(r_grid, density, alpha=0.4, color='steelblue')
    ax.plot(r_grid, density, color='steelblue', linewidth=1.5)
    ax.set_title(f't = {t:.0f} a.u.', fontsize=11)
    ax.set_xlim(0, 25)
    ax.set_ylim(0, density.max() * 1.3 + 0.01)
    ax.set_xlabel('$r / a_0$')
    if idx % 4 == 0:
        ax.set_ylabel('$r^2|\psi|^2$')
    ax.grid(True, alpha=0.2)

plt.suptitle('Time Evolution of Hydrogen Superposition State\n'
             '$|\\psi\\rangle = c_1|1s\\rangle + c_2|2s\\rangle + c_3|3s\\rangle$',
             fontsize=15, y=1.03)
plt.tight_layout()
plt.show()

# Print beat frequencies
omega_12 = abs(E[1] - E[2])
omega_13 = abs(E[1] - E[3])
omega_23 = abs(E[2] - E[3])
print(f"Beat frequencies (atomic units):")
print(f"  ω₁₂ = |E₁ - E₂| = {omega_12:.6f}")
print(f"  ω₁₃ = |E₁ - E₃| = {omega_13:.6f}")
print(f"  ω₂₃ = |E₂ - E₃| = {omega_23:.6f}")
print(f"\nBeat periods:")
print(f"  T₁₂ = 2π/ω₁₂ = {2*np.pi/omega_12:.2f} a.u.")
print(f"  T₁₃ = 2π/ω₁₃ = {2*np.pi/omega_13:.2f} a.u.")
print(f"  T₂₃ = 2π/ω₂₃ = {2*np.pi/omega_23:.2f} a.u.")

In [ ]:
# ── Complete quantum numbers table for hydrogen ─────────────────────

print("=== Complete Hydrogen Quantum Numbers (n = 1 to 4) ===\n")
print(f"{'n':>3} {'l':>3} {'m':>4} {'Orbital':>10} {'E (eV)':>10} "
      f"{'Degeneracy':>12} {'<r> (a₀)':>10}")
print("=" * 58)

total_states = 0
for n in range(1, 5):
    E_n = -13.6 / n**2
    deg = 2 * n**2
    first_in_n = True
    for l in range(n):
        orb_letter = 'spdfg'[l]
        for m in range(-l, l+1):
            total_states += 2  # factor of 2 for spin
            orbital = f'{n}{orb_letter}'
            r_expect = 0.5 * n**2 * (3 - l*(l+1)/n**2)
            if first_in_n:
                print(f"{n:>3} {l:>3} {m:>+4d} {orbital:>10} {E_n:>10.4f} "
                      f"{deg:>12} {r_expect:>10.2f}")
                first_in_n = False
            else:
                print(f"{'':>3} {l:>3} {m:>+4d} {orbital:>10} {'':>10} "
                      f"{'':>12} {r_expect:>10.2f}")
    print("-" * 58)

print(f"\nTotal states (including spin) for n=1 to 4: {total_states}")
print(f"Formula: Σ 2n² = {sum(2*n**2 for n in range(1,5))}  ✓")

## Summary: Year 0 Foundation Complete

### What We Built

Over 336 days and 12 months, we constructed the complete mathematical foundation
for quantum science:

| Month | Topic | Key Tool |
|-------|-------|----------|
| 1-3 | Calculus & ODEs | Derivatives, integrals, differential equations |
| 4-5 | Linear Algebra | Eigenvalues, Hilbert spaces, inner products |
| 6 | Classical Mechanics | Lagrangian, Hamiltonian, Poisson brackets |
| 7 | Complex Analysis | Analytic functions, residues, contour integrals |
| 8 | Electromagnetism | Maxwell's equations, gauge theory |
| 9 | Functional Analysis | Operators, spectra, distribution theory |
| 10 | Scientific Computing | NumPy, SciPy, numerical methods |
| 11 | Group Theory | Symmetries, representations, angular momentum |
| 12 | **Capstone** | **Hydrogen atom: everything together** |

### The Hydrogen Atom Used Every Tool

- **Calculus:** Solving the radial ODE
- **Linear algebra:** Eigenvalue problem $H\psi = E\psi$
- **Special functions:** Laguerre polynomials, spherical harmonics
- **Group theory:** SO(3) symmetry → quantum numbers $l, m$
- **Numerical methods:** Finite-difference sparse eigensolver
- **Visualization:** Orbital plots, energy diagrams

### Ready for Year 1

You now have all the mathematical tools needed to study quantum mechanics
from first principles.  Year 1 begins with the **postulates of quantum mechanics**
and builds up to quantum information and computing.

**The foundation is complete. The quantum journey begins.**

---
*Notebook generated for the SIIEA Quantum Engineering curriculum.*
*License: CC BY-NC-SA 4.0 | Siiea Innovations, LLC*